In [1]:
import os
import sys
os.environ["JAVA_HOME"] = "../data-engineering/.JDK 8"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
import requests, json

# Create SparkSession
spark = SparkSession \
    .builder \
        .appName("GreenDataAnalyticsProject") \
            .getOrCreate()

25/03/29 14:15:08 WARN Utils: Your hostname, Mint-T470 resolves to a loopback address: 127.0.1.1; using 10.46.79.254 instead (on interface wlp4s0)
25/03/29 14:15:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 14:15:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def apiaccess(url):
    try:
        response = requests.get(url)
        geojson_data = response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")

    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")

    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")

    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred with the request: {req_err}")

    except ValueError as json_err:
        print(f"Failed to parse JSON: {json_err}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
    else:
        features = geojson_data.get("features", [])
        json_features = [json.dumps(feature) for feature in features]
        rdd = spark.sparkContext.parallelize(json_features)
        dataframe = spark.read.json(rdd)
        return(dataframe)

In [4]:
api_url = "https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"
df = apiaccess(api_url)

25/03/29 14:15:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df.show()

+--------------------+---+--------------------+-------+
|            geometry| id|          properties|   type|
+--------------------+---+--------------------+-------+
|{[-2.627748803206...|  1|{Yes, NULL, No, N...|Feature|
|{[-2.596813947482...|  2|{Yes, NULL, Yes, ...|Feature|
|{[-2.613994167991...|  3|{No, NULL, Yes, N...|Feature|
|{[-2.578333463205...|  4|{Yes, NULL, Yes, ...|Feature|
|{[-2.595294420664...|  5|{Yes, NULL, Yes, ...|Feature|
|{[-2.558965289247...|  6|{Yes, NULL, No, N...|Feature|
|{[-2.697147645214...|  7|{No, NULL, No, NU...|Feature|
|{[-2.579303486860...|  8|{Yes, NULL, No, N...|Feature|
|{[-2.595417508199...|  9|{Yes, NULL, Yes, ...|Feature|
|{[-2.559380932414...| 10|{Yes, NULL, Yes, ...|Feature|
|{[-2.594190185154...| 11|{Yes, NULL, Yes, ...|Feature|
|{[-2.589439954461...| 12|{Yes, NULL, Yes, ...|Feature|
|{[-2.656900421884...| 13|{No, NULL, No, NU...|Feature|
|{[-2.564144569808...| 14|{Yes, NULL, Yes, ...|Feature|
|{[-2.587283945975...| 15|{Yes, NULL, Yes, ...|F

In [6]:
df = df.select('id',
               col('properties.SiteID').alias('SiteID'),
    col('properties.location').alias('location'),
    col('properties.LocationClass').alias('Location Type'),
    df.geometry.coordinates.alias("coordinates"),
    col('properties.Pollutants').alias('Pollutants'),
    col('properties.Elevation').alias('Elevation')
    )





In [7]:
df.show(1, truncate=False)

+---+------+----------------+-------------+-----------------------------------------+----------+---------+
|id |SiteID|location        |Location Type|coordinates                              |Pollutants|Elevation|
+---+------+----------------+-------------+-----------------------------------------+----------+---------+
|1  |1     |Withywood School|Urban Traffic|[-2.6277488032062446, 51.407745517999835]|BTX NO2   |NULL     |
+---+------+----------------+-------------+-----------------------------------------+----------+---------+
only showing top 1 row



In [8]:
df = df.withColumn("Longitude", col("coordinates")[0])

In [9]:
df = df.withColumn("Latitude", col("coordinates")[1])

In [10]:
from pyspark.sql import functions as F
max_value = df.agg(F.max('Latitude')).collect()[0][0]
min_value = df.agg(F.min('Latitude')).collect()[0][0]

def normalise(original_value):
    try:
        assert max_value > min_value, "max_value must be larger than min_value"
        assert original_value > min_value, "input must be larger than min_value"
        scaled_value = (original_value - min_value) / (max_value - min_value)
    except AssertionError as e:
        return e
        
    else:
        return scaled_value
    

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
normalise_udf = udf(normalise, DoubleType())


In [12]:
df = df.withColumn("Latitude (Normalised)", normalise_udf("Latitude")) # .drop("Latitude")
df.show(truncate=False)

+---+------+-----------------+----------------+-----------------------------------------+----------+---------+-------------------+------------------+---------------------+
|id |SiteID|location         |Location Type   |coordinates                              |Pollutants|Elevation|Longitude          |Latitude          |Latitude (Normalised)|
+---+------+-----------------+----------------+-----------------------------------------+----------+---------+-------------------+------------------+---------------------+
|1  |1     |Withywood School |Urban Traffic   |[-2.6277488032062446, 51.407745517999835]|BTX NO2   |NULL     |-2.6277488032062446|51.407745517999835|0.0909330382676792   |
|2  |2     |Colston Avenue   |Urban Traffic   |[-2.5968139474821794, 51.454556566902696]|NO2       |9.5      |-2.5968139474821794|51.454556566902696|0.2938159127984107   |
|3  |3     |Blackboy Hill    |Urban Traffic   |[-2.613994167991525, 51.46920548941127]  |NO2       |79.2     |-2.613994167991525 |51.4692054

In [13]:
df.toPandas().to_csv("cleanedAirQualityData.csv", index=False)